In [ ]:
%env CUDA_VISIBLE_DEVICES=7

# Постановка задачи

Введем **операцию сравнения** двух векторов f(array_x, array_y) следующим образом:
 1. вектора поэлементно умножаются друг на друга
 2. полученные числа возводятся в квадрат и суммируются

Задан массив **data** размера **(X, Y, Z)**. Для каждого столбца **(data[x, y, :])** необходимо посчитать среднее значение от применения операции сравнения (определена выше) со всеми его соседями в окне по латерали **KxK**.

Написанная функция должна работать для произвольных **X, Y, Z, K**.

Написать функцию таким образом, чтобы изменение способа сравнения f потребовало минимум усилий.

# Код

In [2]:
import numpy as np

In [3]:
X = 1500
Y = 1500
Z = 10
K = 3
data = np.random.rand(X,Y,Z).astype('float32')

# Решение 1 (Неоптимальное)

In [4]:
# Операция сравнения векторов:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

In [5]:
%%time

# Переменная для хранения результатов
res = np.zeros(data.shape[0:2])
k = K // 2 + 1

for x in range(X):
    for y in range(Y):     
        # Для каждого столбца
        col = data[x, y, :]       
        accumulator = 0.0
        for x_shift in range(-k, k + 1):
            for y_shift in range(-k, k + 1):
                # Для каждого соседнего вектора в окне КxК
                if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                    # Сравнить и аккумулировать результат для столбца
                    accumulator += vectors_comparison(col, data[x + x_shift, y + y_shift])
        # Разделить на количество соседей
        x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
        y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
        neighbours_amount = x_window_length * y_window_length - 1
        res[x, y] = accumulator / neighbours_amount

CPU times: user 6min 1s, sys: 50.6 ms, total: 6min 1s
Wall time: 6min 1s


# Решение 1 + Numba

## Numba

 * just-in-time компилятор для python и numpy.
 * Поддерживает некоторое подмножество операций и структур данных для работы с математическими вычислениями.
 * Существует расширение numba-scipy. 

## Вариант 1: Только декоратор

In [6]:
import numba
from numba import njit

In [7]:
X = np.int32(1500)
Y = np.int32(1500)
Z = np.int32(20)
K = np.int32(3)
data = np.random.rand(X,Y,Z).astype('float32')

In [8]:
%%time

@njit
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit
def get_comparison(data, X, Y, Z, K, comparison_function):
    k = K // 2 + 1
    res = np.zeros(data.shape[0:2])
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            accumulator = 0.0
            for x_shift in range(-k, k+1):
                for y_shift in range(-k, k+1):
                    if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                        accumulator += comparison_function(col, data[x + x_shift, y + y_shift])
            x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
            y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
            neighbours_amount = x_window_length * y_window_length - 1
            res[x, y] = accumulator / neighbours_amount
    return res

CPU times: user 289 µs, sys: 4 µs, total: 293 µs
Wall time: 299 µs


In [9]:
%%time
_ = get_comparison(data, X, Y, Z, K, vectors_comparison)

CPU times: user 9.77 s, sys: 5.1 s, total: 14.9 s
Wall time: 9.18 s


In [10]:
%%time
_ = get_comparison(data, X, Y, Z, K, vectors_comparison)

CPU times: user 8.23 s, sys: 40.1 ms, total: 8.27 s
Wall time: 8.24 s


In [11]:
%%timeit
_ = get_comparison(data, X, Y, Z, K, vectors_comparison)

8.21 s ± 3.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Вариант 2: Задать сигнатуры

In [12]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]))
def vectors_comparison_sign(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
      locals={'k': numba.types.int32,
              'res': numba.types.float32[:, :],
              'col': numba.types.float32[:],
              'accumulator': numba.types.float32,
              'neighbours_amount': numba.types.int32,
              'x_window_length': numba.types.int32,
              'y_window_length': numba.types.int32})
def get_comparison_sign(data, X, Y, Z, K):
    k = K // 2 + 1
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            accumulator = 0.0
            for x_shift in range(-k, k+1):
                for y_shift in range(-k, k+1):
                    if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                        accumulator += vectors_comparison(col, data[x + x_shift, y + y_shift])
            x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
            y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
            neighbours_amount = x_window_length * y_window_length - 1
            res[x, y] = accumulator / neighbours_amount
    return res

CPU times: user 629 ms, sys: 8.02 ms, total: 637 ms
Wall time: 636 ms


In [13]:
%%timeit
_ = get_comparison_sign(data, X, Y, Z, K)

8.27 s ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Вариант 3: Использовать дополнительные параметры

In [14]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]),
      fastmath=True)
def vectors_comparison_params(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
      locals={'k': numba.types.int32,
              'res': numba.types.float32[:, :],
              'col': numba.types.float32[:],
              'accumulator': numba.types.float32,
              'neighbours_amount': numba.types.int32,
              'x_window_length': numba.types.int32,
              'y_window_length': numba.types.int32}, 
      parallel=True)
def get_comparison_params(data, X, Y, Z, K):
    k = K // 2 + 1
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            accumulator = 0.0
            for x_shift in range(-k, k+1):
                for y_shift in range(-k, k+1):
                    if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                        accumulator += vectors_comparison(col, data[x + x_shift, y + y_shift])
            x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
            y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
            neighbours_amount = x_window_length * y_window_length - 1
            res[x, y] = accumulator / neighbours_amount
    return res

CPU times: user 806 ms, sys: 12.1 ms, total: 818 ms
Wall time: 811 ms


In [15]:
%%timeit
_ = get_comparison_params(data, X, Y, Z, K)

8.26 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Решение 2: NumPy

In [16]:
xp = np

In [17]:
def matrices_comparison(matrix1, matrix2):
    res = xp.multiply(matrix1, matrix2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def get_comparison(data, X, Y, Z, K, comparison_function):
    k = K // 2 + 1 
    padded_data = xp.pad(data, ((0, k), (k, k), (0, 0)), constant_values=xp.nan)
    result = []      
    for x_i in range(k):
        for y_i in range(-k+1, k):
            if (x_i == 0) and (y_i <= 0):
                continue            
            # Сравнить матрицу и матрицу со сдвигом
            comparison = comparison_function(data, padded_data[x_i:x_i+X, k+y_i:k+y_i+Y])
            # Сдвинуть результат сравнения
            shifted_comparison = xp.pad(
                comparison[:X-x_i, max(0, -y_i):min(Y, Y-y_i)], 
                ((x_i, 0), (max(0, y_i), -min(0, y_i))), 
                constant_values=xp.nan
            )
            result.extend([comparison, shifted_comparison])
    return xp.nanmean(result)

In [18]:
%%timeit
_ = get_comparison(data, X, Y, Z, K, matrices_comparison)

2.45 s ± 2.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Решение 2: CuPy

In [19]:
import cupy as cp
xp = cp

In [20]:
def matrices_comparison(matrix1, matrix2):
    res = xp.multiply(matrix1, matrix2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def get_comparison(data, X, Y, Z, K, comparison_function):
    k = K // 2 + 1 
    padded_data = xp.pad(data, ((0, k), (k, k), (0, 0)), constant_values=xp.nan)
    result = []    
    for x_i in range(k):
        for y_i in range(-k+1, k):
            if (x_i == 0) and (y_i <= 0):
                continue            
            comparison = comparison_function(data, padded_data[x_i:x_i+X, k+y_i:k+y_i+Y])
            shifted_comparison = xp.pad(
                comparison[:X-x_i, max(0, -y_i):min(Y, Y-y_i)], 
                ((x_i, 0), (max(0, y_i), -min(0, y_i))), 
                constant_values=xp.nan
            )
            result.extend([comparison, shifted_comparison])
    return xp.nanmean(cp.asarray(result))

In [21]:
%%time
data_n = cp.asarray(data)

CPU times: user 4.84 ms, sys: 3.82 s, total: 3.82 s
Wall time: 4.1 s


In [22]:
%%time 
_ = get_comparison(data_n, X, Y, Z, K, matrices_comparison)

CPU times: user 530 ms, sys: 7.81 ms, total: 537 ms
Wall time: 536 ms


In [23]:
%%timeit
_ = get_comparison(data_n, X, Y, Z, K, matrices_comparison)

23 ms ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### test: 

<img src="images/1_time_exec.jpg">